In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_ablation_baselinegp as file

In [9]:
# Args setup
ntrial = 50
n = 100
ntest = 100
d = 5
noise = 0.5
niter = 1000
error_samples = 100
gp_samples = 1000
kernel = "gammaexp"

In [10]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [11]:
# Submitting jobs
futures = []
for seed in range(ntrial):
            f = client.submit(file.main,seed,n,ntest,d,noise,
                             error_samples = error_samples, gp_samples = gp_samples,
                             niter = niter, kernel = kernel)
            futures += [f]

In [12]:
# Check on futures
futures

[<Future: pending, key: main-df5600526210253a924d0cc480cecb3b>,
 <Future: pending, key: main-1c25a9d22c4a1d676aa6e90cc2c26561>,
 <Future: pending, key: main-b349a7d3428326eb27b97129b631afa5>,
 <Future: pending, key: main-4f3263a77e69848bc7319c094801c799>,
 <Future: pending, key: main-982d599686582bb16df0ed316c1a92a3>,
 <Future: pending, key: main-40634e47802eb16f03ebc0cf594674f2>,
 <Future: pending, key: main-71553e5a0f9ebc9c0d6b6aa77eed4c06>,
 <Future: pending, key: main-a732d186f502806711f6ff2ae1619207>,
 <Future: pending, key: main-cfc2562a60b68d048b3224555ef6241b>,
 <Future: pending, key: main-570e437015974ed32d5f6681451c5607>,
 <Future: pending, key: main-a2fcb39e60919d10c274ffb295d86907>,
 <Future: pending, key: main-1b8d26511ef46c117557fb183125677a>,
 <Future: pending, key: main-44dd78d0c69712a65b2fad5a27efddac>,
 <Future: pending, key: main-2fe86b428db7b2c2eeb9a75e1a694bbd>,
 <Future: pending, key: main-adf8699c5ac27b3df40268d90af6c182>,
 <Future: pending, key: main-83384e4ec39

In [13]:
# Getting results
results = client.gather(futures)

In [14]:
# Closing client
client.close()
cluster.close()

In [15]:
# Saving results
torch.save(obj = results,
           f = "ablation_baselinegp_ntrial={0}_n={1}_d={2}_noise={3}_kernel={4}.pt".format(ntrial,n,d,noise,kernel))